In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tqdm import tqdm, tqdm_notebook

#warnings.filterwarnings('ignore')
# BEWARE, ignoreing warnings is not always a good idea
# I am doing it for presentation

# Private and Encrypted AI - Credit Approval Application

This notebook is meant for my exploratory development of en encrypted federated deep learning approach.
I will develop a final model in a separate folder.

### Glossary
1. [Data Preparation & Setup](#data_prep)
2. [Classical Deep Learning](#classical_dl)
3. [Federated Deep Learning](#federated_dl)<br>
    3.1 [Model Averaging with Trusted Aggregator](#fl_model_avg)
4. [Encrypted Deep Learning](#encrypted_dl)<br>
   4.1 [Secured Multi-Party Computation (SMPC)](#smpc) <br>
   4.2 [Encrypted Gradient Averaging](#fl_encrypt_avg)<br>
   4.3 [Differential Privacy for DL](#dp_dl)
   
<hr>

_Notes_ <br>This project was inspired by lectures of [Andrew Trask](https://iamtrask.github.io/) in the [Private AI Scholarship Challenge on Udacity](https://www.udacity.com/facebook-AI-scholarship). Furthermore, segments of the code are inspired by the [PySyft tutorials on GitHub](https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials); an excellent resource for people starting off with Private AI. 

<a id='data_prep'></a>
## Data Preparation
- only using non-NaN values. I drop NaN values because the dataset is not very big regardless, and we are not dropping very many values.
- Convert binary variables to a numeric representation, and one-hot-encode categorical variables. We do not want to use label encoder since a label encoder would make it 

In [2]:
cols = [ f"A{i}" for i in range(1,16)]
cols.append('label')

In [3]:
df = pd.read_csv('data/crx.data', names=cols)\
    .replace(to_replace='?', value=np.nan).dropna()
print(df.shape, "\n ------- \n")
print(df.head(2))

(653, 16) 
 ------- 

  A1     A2    A3 A4 A5 A6 A7    A8 A9 A10  A11 A12 A13    A14  A15 label
0  b  30.83  0.00  u  g  w  v  1.25  t   t    1   f   g  00202    0     +
1  a  58.67  4.46  u  g  q  h  3.04  t   t    6   f   g  00043  560     +


### Data Analysis

Let's check out what this data looks like first, so that we have an idea of what we are dealing with. In true encrypted, federated learning we would not have this luxury though...

In [4]:
def to_binary(df, col):
    u = df[col].unique()
    mapping =dict(zip(u, [i for i in range(0,len(u))]))
    return df[col].map(mapping)

In [5]:
df.A1.head()

0    b
1    a
2    a
3    b
4    b
Name: A1, dtype: object

In [6]:
#convert to float
for col in ['A2', 'A3', 'A8', 'A11', 'A14', 'A15']:
    df[col] = df[col].astype(float)
    
#binarize
for col in ['A1', 'A9', 'A10', 'A12', 'label']:
    df[col] = to_binary(df, col)
    
onehot_cols = ['A4', 'A5', 'A6', 'A7', 'A13']

#perform one hot encoding, and drop original columns
df  = df.join(pd.get_dummies(df[onehot_cols], dtype=int))\
                                .drop(onehot_cols, axis=1)

In [7]:
set(df.dtypes) #check that we have the data types we expect, no object types

{dtype('int64'), dtype('float64')}

In [8]:
#distribution of numeric-only columns
df[['A2', 'A3', 'A8', 'A11', 'A14', 'A15']].describe().iloc[1:, :10].round(3)

,A2,A3,A8,A11,A14,A15
mean,31.504,4.830,2.244,2.502,180.360,1013.761
std,11.838,5.027,3.371,4.968,168.297,5253.279
min,13.750,0.000,0.000,0.000,0.000,0.000
25%,22.580,1.040,0.165,0.000,73.000,0.000
50%,28.420,2.835,1.000,0.000,160.000,5.000
75%,38.250,7.500,2.625,3.000,272.000,400.000
max,76.750,28.000,28.500,67.000,2000.000,100000.000


In [9]:
df.head(2) #double check what our DF looks like

,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_ff,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s
0,0,30.83,0.00,1.25,0,0,1.0,0,202.0,0.0,...,0,0,0,0,0,1,0,1,0,0
1,1,58.67,4.46,3.04,0,0,6.0,0,43.0,560.0,...,0,1,0,0,0,0,0,1,0,0


### Simulate Real People's Data

To illustrate how this model would work in real life, I want to simulate this data belonging to people. I am generating random names to be associated with each row. I know that this is not an ideal example since I am in fact starting with the data all collated on my computer with peoples names and data being directly exposed. Not private at all...

In [10]:
import names #used to get random names
names.get_first_name()+' ' +names.get_last_name() #call random name

'Jerry Canales'

In [11]:
users = []
used_names = set()
for idx in range(len(df)):
    name = names.get_first_name()+' ' +names.get_last_name()
    while name in used_names:
        name = names.get_first_name()+' ' +names.get_last_name()
        
    used_names.add(name)
    users.append(name)

In [12]:
df['name'] = users
df.head(2)

,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s,name
0,0,30.83,0.00,1.25,0,0,1.0,0,202.0,0.0,...,0,0,0,0,1,0,1,0,0,Mandi Lerma
1,1,58.67,4.46,3.04,0,0,6.0,0,43.0,560.0,...,1,0,0,0,0,0,1,0,0,Rebecca Caldwell


In [13]:
#get features and labels as numpy arrays which we can convert to tensors
features = df.drop(['label', 'name'], axis=1).values.astype(float)
labels = df['label'].values.astype(float)


#normalize
sclr = MinMaxScaler()
features = sclr.fit_transform(features)

In [14]:
#save features and labels for future use
np.save('data/features', features)
np.save('data/labels', labels)

#save labels where shape is (1,2)
labels=pd.get_dummies(df['label']).values.astype(float)
np.save('data/labels_dim', labels)


_Please Note_ <br>
Normalization is not necessary per se for any machine learning algorithm, but it is recommended for deep learning for training purposes. Read more [here](https://datascience.stackexchange.com/a/13221/60648).

## Model Development
I am using PyTorch to create a neural network to classify whether someone is accepted for credit or not. PyTorch integrates will with PySyft, the package used to encrypt our deep learning model

In [15]:
import copy
from torch import nn
from torch import optim
import torch.nn.functional as F
import syft as sy
import torch as th
th.manual_seed(42) #so that dropout affects same layers

data = th.tensor(features, dtype=th.float32, requires_grad=True)
target = th.tensor(labels, dtype=th.float32, requires_grad=False).reshape(-1,2)

class Model(nn.Module):
    '''
    Neural Network Example Model
    
    Attributes
    :hidden_layers (nn.ModuleList) - hidden units and dimensions for each layer
    :output (nn.Linear) - final fully-connected layer to handle output for model
    :dropout (nn.Dropout) - handling of layer-wise drop-out parameter
    
    Functions
    :forward - handling of forward pass of datum through the network.
    '''
    def __init__(self, args):
        super(Model, self).__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(args.in_size,
                                                      args.hidden_layers[0])])

        #create hidden layers
        layer_sizes = zip(args.hidden_layers[:-1], args.hidden_layers[1:]) 
        #gives input/output sizes for each layer
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        self.output = nn.Linear(args.hidden_layers[-1], args.out_size)
        self.dropout = None if args.drop_p is None \
                                            else nn.Dropout(p=args.drop_p)
        
    def forward(self, x):
        x = x.view(-1, args.in_size)
        for each in self.hidden_layers:
            x = F.relu(each(x)) #apply relu to each hidden node
            
            if self.dropout is not None:
                x = self.dropout(x) #apply dropout
                
        x = self.output(x) #apply output weights
        
        if args.activation is None:
            return x
        
        return args.activation(x, dim=args.dim) #apply activation log softmax

/home/mkucz/p_venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mkucz/p_venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mkucz/p_venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mkucz/p_venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

<a id='classical_dl'></a>
## Classical Deep Learning
Here we train our network on data that is not distributed (therefore this is not yet a federated or encrypted problem). However, this exercise is useful in showing how we can transition from traditional deep learning to federated deep learning.

First create a dataset of batch size one. This is realistic since most people would only have their own credit score data. This might be different if we decide to use a secure or trusted third party to manage parts of the data, but we don't trust the credit rating company with our data.

In [32]:
class Arguments():
    def __init__(self, in_size, out_size, hidden_layers,
                       activation=F.softmax, dim=-1):
        self.batch_size = 1
        self.drop_p = None
        self.epochs = 300
        self.lr = 0.001
        self.in_size = in_size
        self.out_size = out_size
        self.hidden_layers = hidden_layers
        self.precision_fractional=10
        self.activation = activation
        self.dim = dim

In [33]:
dataset = [(data[i], target[i].reshape(1,2)) for i in range(len(data))]

#instantiate model
in_size = data[0].shape[0]
out_size = 2
hidden_layers=[32,15,8]

In [34]:
_data, _target = dataset[0]
_data, _target

(tensor([0.0000, 0.2711, 0.0000, 0.0439, 0.0000, 0.0000, 0.0149, 0.0000, 0.1010,
         0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000],
        grad_fn=<SelectBackward>), tensor([[1., 0.]]))

In [35]:
def train(model, datasets, criterion):
    #use a simple stochastic gradient descent optimizer
    #define optimizer for each model
    optimizer = optim.SGD(params=model.parameters(), lr=args.lr)
    steps=0
    model.train() #training mode
    for e in range(1, args.epochs+1):
        running_loss=0
        for ii, (data,target) in enumerate(datasets): #iterates over pointers to remote data
            steps+=1
            optimizer.zero_grad()#zero out gradients so that one forward pass doesnt pick up previous forward's gradients
            outputs = model.forward(data) #make prediction
            outputs = outputs.reshape(1,-1) #get shape of (1,2) as we need at least two dimension
            loss = criterion(outputs, target)

            loss.backward()
            optimizer.step()
            
            #print(f"step: {steps}", loss.item())
            running_loss+=loss.item()

        if e%10==0:
            print(f'Epoch: {e} \tLoss: {running_loss/len(datasets):.6f}')
        running_loss=0


In [36]:
args = Arguments(in_size, out_size, hidden_layers, activation=F.softmax, dim=1)
base_model = Model(args)

In [120]:
model = copy.deepcopy(base_model) #exact replica of base model
train(model, dataset, nn.MSELoss())

Epoch: 1 	Loss: 0.247287
Epoch: 2 	Loss: 0.246489
Epoch: 3 	Loss: 0.245740
Epoch: 4 	Loss: 0.244993
Epoch: 5 	Loss: 0.244216
Epoch: 6 	Loss: 0.243396
Epoch: 7 	Loss: 0.242531
Epoch: 8 	Loss: 0.241591
Epoch: 9 	Loss: 0.240556
Epoch: 10 	Loss: 0.239403
Epoch: 11 	Loss: 0.238106
Epoch: 12 	Loss: 0.236645
Epoch: 13 	Loss: 0.234978
Epoch: 14 	Loss: 0.233058
Epoch: 15 	Loss: 0.230859
Epoch: 16 	Loss: 0.228302
Epoch: 17 	Loss: 0.225384
Epoch: 18 	Loss: 0.222019
Epoch: 19 	Loss: 0.218137
Epoch: 20 	Loss: 0.213731
Epoch: 21 	Loss: 0.208719
Epoch: 22 	Loss: 0.203057
Epoch: 23 	Loss: 0.196793
Epoch: 24 	Loss: 0.189950
Epoch: 25 	Loss: 0.182593


KeyboardInterrupt: 

We can also use PyTorch's `Dataset` class to make the processing of data a little easier, but for the purpose of this example it will not give any clear benefits. If you would like to read more about PyTorch's abstract `Dataset` class [read here](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html), with another example [here](https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel). Generally speaking, using `Dataset` and `DataLoader` makes the handling of training and testing data much easier.

In [37]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
n_train_items = int(len(dataset)*0.7)
n_test_items = len(dataset) - n_train_items

train_dataset = TensorDataset(data[:n_train_items], target[:n_train_items])
test_dataset = TensorDataset(data[:n_train_items], target[:n_train_items])

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
#this gives us an identical implementation, but split up into train/test set

In [38]:
%%time
#training loss will look a little different since the dataset is shuffled
model = copy.deepcopy(base_model)
train(model, train_loader, nn.MSELoss())

Epoch: 10 	Loss: 0.245377
Epoch: 20 	Loss: 0.239523
Epoch: 30 	Loss: 0.224637
Epoch: 40 	Loss: 0.181348
Epoch: 50 	Loss: 0.120974
Epoch: 60 	Loss: 0.099086
Epoch: 70 	Loss: 0.093512
Epoch: 80 	Loss: 0.091027
Epoch: 90 	Loss: 0.089384
Epoch: 100 	Loss: 0.088098
Epoch: 110 	Loss: 0.087058
Epoch: 120 	Loss: 0.086190
Epoch: 130 	Loss: 0.085509
Epoch: 140 	Loss: 0.084796
Epoch: 150 	Loss: 0.084161
Epoch: 160 	Loss: 0.083559
Epoch: 170 	Loss: 0.082943
Epoch: 180 	Loss: 0.082355
Epoch: 190 	Loss: 0.081759
Epoch: 200 	Loss: 0.081160
Epoch: 210 	Loss: 0.080565
Epoch: 220 	Loss: 0.079908
Epoch: 230 	Loss: 0.079232
Epoch: 240 	Loss: 0.078528
Epoch: 250 	Loss: 0.077749
Epoch: 260 	Loss: 0.076891
Epoch: 270 	Loss: 0.076085
Epoch: 280 	Loss: 0.075072
Epoch: 290 	Loss: 0.074087
Epoch: 300 	Loss: 0.073109
CPU times: user 1min 8s, sys: 2.1 s, total: 1min 10s
Wall time: 1min 9s


In [39]:
def test(model, dataloader, criterion):
    print(f'Testing')
    steps = 0
    model.eval()  # training mode

    pred = []
    true = []
    running_loss=0.
    for ii, (data, target) in tqdm_notebook(enumerate(dataloader),
                                            unit='datum', desc='testing',
                                            total=len(dataloader)):
        # iterates over pointers to remote data
        steps += 1
        outputs = model.forward(data) #make prediction
        outputs = outputs.reshape(1,-1)
        #get shape of (1,2) as we need at least two dimension
        loss = criterion(outputs, target)
        
        _, y_pred = th.max(outputs, 1)
        _, y_true = th.max(target, 1)

        pred.append(y_pred.item())
        true.append(y_true.item())

        # get loss from remote worker and unencrypt

        running_loss += loss.item()
    print('Testing Loss: {:.6f}'.format(running_loss/len(dataset)))
    
    return pred, true
    

In [40]:
y_pred, y_true = test(model, test_loader, nn.MSELoss())

Testing



Testing Loss: 0.053298


In [43]:
cnf_mtx = confusion_matrix(y_pred, y_true).astype(int)
print(cnf_mtx)

tp = cnf_mtx[1][1]
tn = cnf_mtx[0][0]
total = cnf_mtx.sum()

print(f"accuracy: {(tp+tn)/total:.2f}")
print(f"recall: {tp/sum(y_true):.2f}")
print(f"precision: {tp/sum(y_pred):.2f}")

[[197  27]
 [ 15 218]]
accuracy: 0.91
recall: 0.89
precision: 0.94


Now we have a credit application model that is training on our data. However, this is by no means yet federated learning. The implementation above simply trains a model with a batch size of 1. We will federate the model in the upcoming section [here]().

Generally, the model looks pretty solid across the test sets. I will save the model parameters here so that we can use them in our federated or encrypted models (so as not to train it all from scratch again).

Also, it took 70 seconds to train this model which amounts to about 0.233 seconds per epoch.